In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/10707/')
!pip install import-ipynb
import import_ipynb

!pip install optuna
import optuna
import os
import numpy as np
import torch
import logging

Mounted at /content/drive
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=7847d688e6f761c7d849e35eafe5a29ac4260ed9627d1ce18f5525177462190f
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
     |████████████████████████████████| 308 kB 4.2 MB/s 
     |████████████████████████████████| 81 kB 8.1 MB/s 
     |████████████████████████████████| 210 kB 48.7 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 113 kB 46.3 MB/s 
     |████████████████████████████████| 150 kB 47.4 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=fe3137857cd96754c3cda227b20be7ebec3dee485a2394c340f0a71ee15bf8b4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully

In [2]:
notebooks = ['train']

for notebook in notebooks:
  source_path_file = '/content/drive/My Drive/Colab Notebooks/10707/{}.ipynb'.format(notebook)
  source_path_file = source_path_file.replace(' ', '\\ ')
  !cp $source_path_file '/content' # to copy the file from drive to colab
  !rsync -aP $source_path_file '/content/{}.ipynb'.format(notebook)

from train import TrainCMB

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `rsync -aP $source_path_file '/content/{}.ipynb'.format(notebook)'
importing Jupyter notebook from train.ipynb
Mounted at /content/drive
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `rsync -aP $source_path_file '/content/{}.ipynb'.format(notebook)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `rsync -aP $source_path_file '/content/{}.ipynb'.format(notebook)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `rsync -aP $source_path_file '/content/{}.ipynb'.format(notebook)'
importing Jupyter notebook from networks.ipynb
Mounted at /content/drive
sending incremental file list
utils.ipynb
          6,961 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=0/1)
importing Jupyter notebook from utils.ipynb
Mounted at /content/drive
importing Jupyter notebook from pipeline.ipynb
Mounted at /content/drive


In [3]:
class BaseOption(object):
    def __init__(self):
        self.is_train = True
        self.n_epochs = 150
        self.latest = 0
        self.beta1 = 0.5
        self.beta2 = 0.999
        self.epoch_decay = 100
        self.GAN_type = 'LSGAN'
        self.lambda_FM = 10
        self.lr = 0.0002
        self.n_D = 2   
        self.no_shuffle = False
        self.gpu_ids = 0
        self.data_format_input = 'npy'
        self.data_format_target = 'npy'
        self.dataset_dir = '/content/drive/My Drive/Colab Notebooks/10707/Data/debugging'
        self.checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/10707/checkpoint'
        self.input_ch = 1
        self.saturation_lower_limit_input = 1
        self.saturation_upper_limit_input = 200
        self.saturation_lower_limit_target = -3000
        self.saturation_upper_limit_target = 3000
        self.batch_size = 1
        self.dataset_name = 'CMB'
        self.data_type = 32
        self.image_mode = 'png'
        self.n_downsample = 4
        self.n_residual = 9
        self.n_workers = 1
        self.norm_type = 'InstanceNorm2d'
        self.padding_type = 'zero'
        self.padding_size = 0
        self.max_rotation_angle = 0
        self.val_during_train = False
        self.report_freq = 10
        self.save_freq = 10000
        self.display_freq = 100
        self.format = 'png'
        self.n_df = 64
        self.flip = 64
        self.n_gf = 32
        self.output_ch = 1

        if self.data_type == 16:
            self.eps = 1e-4
        elif self.data_type == 32:
            self.eps = 1e-8

        dataset_name = self.dataset_name
        checkpoint_dir = self.checkpoint_dir
        os.makedirs(os.path.join(checkpoint_dir, dataset_name, 'Image', 'Train'), exist_ok=True)
        os.makedirs(os.path.join(checkpoint_dir, dataset_name, 'Image', 'Test'), exist_ok=True)
        os.makedirs(os.path.join(checkpoint_dir, dataset_name, 'Model'), exist_ok=True)

        if self.is_train:
            self.image_dir = os.path.join(checkpoint_dir, dataset_name, 'Image/Train')
        else:
            self.image_dir = os.path.join(checkpoint_dir, dataset_name, 'Image/Test')
        self.model_dir = os.path.join(checkpoint_dir, dataset_name, 'Model')

class TrainOption(BaseOption):
    def __init__(self):
        super(TrainOption, self).__init__()
        self.is_train = True
        self.n_epochs = 150
        self.latest = 0
        self.beta1 = 0.5
        self.beta2 = 0.999
        self.epoch_decay = 100
        self.GAN_type = 'LSGAN'
        self.lambda_FM = 10
        self.lr = 0.0002
        self.n_D = 2   
        self.no_shuffle = False

In [31]:
import copy

base_opt = BaseOption()
opt = TrainOption()
opt.dataset_name = 'CMB' 
opt.input_ch =  1 
opt.n_downsample = 3
opt.n_residual = 5
opt.n_epochs =  1
opt.norm_type = 'InstanceNorm2d'
opt.batch_size =  128
opt.report_freq =  1
opt.beta1 = 0.5
opt.save_freq =  281
opt.lr = 0.0002
opt.display_freq  = 10000000000
opt.val_during_train = True


logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("/content/drive/My Drive/Colab Notebooks/10707/studies/hyperparam_search.log", mode="w"))
optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.



#put any args to search in here
def exp_opts(trial, opt):


    #########
    ### put any param to search here
    ##########
    opt.lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    opt.n_downsample = trial.suggest_int('n_downsample', 2, 4)
    opt.n_residual = trial.suggest_int('n_residual', 2, 8)

    #############
    #### also make your additions here when you add opts !!
    ##############
    out_str = 'MIN VAL ERROR n_downsample{}_n_residual{}_lr{}_ IS '.format(opt.n_downsample,opt.n_residual,opt.lr)
    return opt, out_str

def objective(trial, opts):
    # Get the hparams suggested by optuna
    opt, out_str = exp_opts(trial, opts)

    val_opt = copy.copy(opt)
    val_opt.is_train = False
    val_opt.is_val = True
    val_opt.no_shuffle = True

    model = TrainCMB([opt, val_opt])
    model(opt)
    
    #first col indicates ll loss, second col indicates power spectrum loss
    val_loss = torch.tensor(model.val_loss).reshape((-1,2))

    ll     = val_loss[:,0]
    ps     = val_loss[:,1]
    ps_    = (10**7) * ps
    
    min_idx = torch.argmin(ps_+ll)
    min_val_loss = torch.min(ps_+ll)
    
    print('{} {} . LL loss is {} and PS loss is {}'.format(out_str, min_val_loss, ll[min_idx], ps[min_idx]))
    return min_val_loss

# If a study already exists, load that one
study_name = 'study'
study = optuna.create_study(
        direction='minimize', 
        study_name=study_name,
        # storage='sqlite:///content/drive/My Drive/Colab Notebooks/10707/studies/{}.db'.format(study_name),
        load_if_exists=True # If file exists, load it and resume instead
    )


###### put how many trials here!!
logger.info("Start optimization.")
study.optimize(lambda trial: objective(trial, opt), n_trials=2)

with open("/content/drive/My Drive/Colab Notebooks/10707/studies/hyperparam_search.log") as f:
    assert f.readline() == "Start optimization.\n"
    assert f.readline().startswith("Finished trial#0 with value:")

Generator(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (11): ReLU(inplace=True)
    (12): ResidualBlock(
      (block): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1 [100.0%] Current_step: 1 D_loss: 3.5022  G_loss: 46.97  L2_loss: 0.98882



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1/1 [00:00<00:00,  3.37it/s]

LL is  tensor(1.0308, device='cuda:0')
Total time taken:  0:00:00.312471
MIN VAL ERROR n_downsample3_n_residual3_lr0.0002528011552229213_ IS  1.0307769796131194 . LL loss is 1.0307769775390625 and PS loss is 2.074056980197792e-09


...

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1 [100.0%] Current_step: 1 D_loss: 4.0448  G_loss: 49.258  L2_loss: 1.0137



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1/1 [00:00<00:00,  3.30it/s]

LL is  tensor(1.0505, device='cuda:0')
Total time taken:  0:00:00.313329
MIN VAL ERROR n_downsample3_n_residual6_lr8.710006264994554e-05_ IS  1.0505207789990494 . LL loss is 1.0505207777023315 and PS loss is 1.296717785307942e-09


AssertionError: ignored